# Inverse Compton Scattering

At the low energy limit, Tracy calculated an approximate expression for dN/(dt deps1), which is the number of photons per scattered energy eps1 per unit time. This expression is presented as an integrand, integrated over the CMB spectrum.

In [4]:
import numpy as np

from scipy import integrate

import darkhistory.physics as phys

In [98]:
def icsspec_loweng(eleckineng_arr, photeng_arr, rs):
    
    gamma_arr = 1 + eleckineng_arr/phys.me
    beta_arr = np.sqrt(1-1/(gamma_arr**2))
    
    def loweng_integrand(CMBeng, eleckineng, photeng):
        """

        Parameters
        ----------

        eleckineng_arr : float
            Kinetic energy of the electron.
        photeng_arr : float
            Energy of the scattered photon. 
        CMBeng : float
            Energy of the CMB photon (to be integrated over)
        """
        gamma = 1 + eleckineng/phys.me
        beta = np.sqrt(1 - 1/(gamma**2))

        def prefac(CMBeng): 
            return phys.c*(3/16)*(
                phys.thomson_xsec/((gamma**3)*(beta**2)*(CMBeng**2))
            )

        def integrand(CMBeng, photeng):
            """
            CMBeng is a double, photeng can be an ndarray.
            """

            photenghigh = (
                CMBeng*(1+beta**2)/(beta**2)*np.sqrt((1+beta)/(1-beta))
                + (2/beta)*np.sqrt((1-beta)/(1+beta))*photeng
                - ((1-beta)**2)/(beta**2)*np.sqrt((1+beta)/(1-beta))*(
                     photeng**2/CMBeng
                )
                + 2/(gamma*beta**2)*photeng*np.log(
                     (1-beta)/(1+beta)*photeng/CMBeng
                )
            )

            photenglow = (
                - CMBeng*(1+beta**2)/(beta**2)*np.sqrt((1-beta)/(1+beta))
                + (2/beta)*np.sqrt((1+beta)/(1-beta))*photeng
                + (1+beta)/(gamma*beta**2)*(photeng**2/CMBeng)
                - 2/(gamma*beta**2)*photeng*np.log(
                     (1+beta)/(1-beta)*photeng/CMBeng
                 )
            )

            outval = np.where(photeng > CMBeng, photenghigh, photenglow)
            # Zero out parts where photeng exceeds its theoretical max value.
            outval = np.where(photeng < gamma**2*(1+beta)**2*CMBeng, outval, np.zeros(outval.size))
            return outval

        return prefac(CMBeng)*integrand(CMBeng, photeng)
    
    def full_loweng_integrand(CMBeng, eleckineng, photeng):
        
        return loweng_integrand(CMBeng, eleckineng, photeng)*phys.CMB_spec(CMBeng, phys.TCMB(rs))
    
    lowlim = np.array(
        [
            (1.-beta)/(1.+beta)*photeng_arr
            for beta in beta_arr
        ]
    )
    
    upplim = np.array(
        [
            (1.+beta)/(1.-beta)*photeng_arr
            for beta in beta_arr
        ]
    )
    
    # Zero out where the CMB spectrum is already set to zero.
    upplim = np.where(upplim < 500.*phys.TCMB(rs), upplim, 
                      500.*phys.TCMB(rs)*np.ones(upplim.shape))
    print(lowlim)
    print(upplim)
    
    return np.array(
        [
            [integrate.quad(full_loweng_integrand, lowlim[i,j], upplim[i,j],
                            args=(eleceng, photeng), epsabs=0, epsrel=1e-3)[0]
                for j,photeng in zip(np.arange(photeng_arr.size),photeng_arr)]
            for i,eleceng in zip(np.arange(eleckineng_arr.size),eleckineng_arr)
        ]
    )
    
    
    

In [100]:
testeleceng_arr = np.array([2e7, 6.3043e9])
testphoteng_arr = np.array([1.65e-2, 3.784, 14019.292910, 3.430934e10])
a = icsspec_loweng(testeleceng_arr, testphoteng_arr, 10000)
print('****************')
print(a)

[[  2.56109181e-06   5.87343721e-04   2.17604220e+00   5.32541634e+06]
 [  2.70969135e-11   6.21422550e-09   2.30230041e-05   5.63440738e+01]]
[[  106.30231963  1175.          1175.          1175.        ]
 [ 1175.          1175.          1175.          1175.        ]]
****************
[[  2.80507671e-05   9.59372412e-04   1.85770062e-04  -0.00000000e+00]
 [  2.99460086e-10   1.02616542e-08   1.17579967e-08   2.56813187e-19]]
